# This is the python template for Assignment 04.  
- You must use this template.  
- You must not change any signatures of the methods, only edit the sections indicated with "Write your code here."  
- The return of every function has to be in the right format, otherwise this is a desk reject.  
- Plagiarism leads to failing the assignment!  
- We will terminate the script after 10 min, try to use efficient algorithms.

In [1]:
import pandas as pd

In [2]:
def get_name():
    return "Ali Salaheldin Ali Ahmed"
def get_matriculationnumber():
    return 7043295

## Useful information:

The structure of a CART is a dict. Use the same names as shown in the example, using other names makes your format invalid and leads to a desk reject.

In [3]:
cart = { "name":"X", "mean":456, "split_by_feature": "aes", "error_of_split": 0.0,
        "successor_left": { "name":"XL", "mean":1234, "split_by_feature": None, "error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          },
        "successor_right":{ "name":"XR", "mean":258, "split_by_feature": None,"error_of_split":None,
                           "successor_left":None,
                           "successor_right":None
                          }
       }

The names of the features must be used as defined in this list, using other names makes your format invalid and leads to a desk reject.

In [4]:
features = ["secompress", "encryption", "aes", "blowfish", "algorithm", "rar", "zip", "signature",
            "timestamp", "segmentation", "onehundredmb", "onegb"]

# Task 1: Create a CART

In [23]:
# Write your helper functions here, if needed
class Node:
    def __init__(self, name, mean, feature=None, error_of_split=None, left=None, right=None):
        self.name = name
        self.mean = mean
        self.feature = feature
        self.error = error_of_split
        self.left = left
        self.right = right

    def __str__(self):
        if self.feature is None:
            return f"{self.name}({self.mean})"
        return f"{self.name}({self.feature}, m={self.mean}, e={self.error})"

    def as_dict(self):
        return {
            "name": self.name,
            "mean": self.mean,
            "split_by_feature": self.feature,
            "error_of_split": self.error,
            "successor_left": self.left.as_dict() if self.left is not None else None,
            "successor_right": self.right.as_dict() if self.right is not None else None,
        }


class Split:
    def __init__(self, X: pd.DataFrame, y: pd.Series, feature: str):
        self.feature = feature

        f = X[feature] == 1
        self.data_with = X[f], y[f]
        self.mean_with = self.data_with[1].mean()

        self.data_without = X[~f], y[~f]
        self.mean_without = self.data_without[1].mean()

        self.error = (
            ((self.data_with[1] - self.mean_with)**2).sum() + 
            ((self.data_without[1] - self.mean_without)**2).sum()
        )

    def is_valid(self):
        return len(self.data_with[1]) > 0 and len(self.data_without[1]) > 0
    
    def __lt__(self, other):
        return self.error < other.error

    def __eq__(self, other):
        return self.error == other.error

    def __str__(self):
        return f"Split(feature={self.feature}, error={self.error})"

class CARTStep:
    def __init__(self, X: pd.DataFrame, y: pd.Series, node: Node, path: set[str] = None):
        self.X = X
        self.y = y
        self.node = node
        self.path = path if path is not None else set()

    def split_by(self, split: Split):
        self.node.feature = split.feature
        self.node.error = split.error
        self.node.left = Node(
            f"{self.node.name}L",
            split.mean_with,
        )
        self.node.right = Node(
            f"{self.node.name}R",
            split.mean_without,
        )

        (Xl, yl) = split.data_with
        (Xr, yr) = split.data_without
        succ_path = self.path | {split.feature}
        return (
            CARTStep(Xl, yl,
                     node=self.node.left,
                     path=succ_path),
            CARTStep(Xr, yr,
                     node=self.node.right,
                     path=succ_path),
        )

def print_cart(cart):
    queue = [cart]
    level = 1
    while queue:
        node = queue.pop(0)
        if node is not None:
            if node["split_by_feature"] is None:
                print(f"{node['name']}(m={node['mean']})", end="")
            else:
                print(f"{node['name']}({node['split_by_feature']}, {node['mean']}, e={node['error_of_split']})", end="")
            queue.extend((node['successor_left'],node['successor_right']))
        print("\t", end="")
        if (level := level - 1) == 0:
            print("\n", end="")
            level = len(queue)

In [28]:
ALL_FEATURES = {"secompress", "encryption", "aes", "blowfish", "algorithm", "rar", "zip", "signature",
                "timestamp", "segmentation", "onehundredmb", "onegb"}


def get_cart(sample_set_csv):
    # The sample_set_csv is a file path to a csv data, this can be imported into a dataframe
    df = pd.read_csv(sample_set_csv)
    # TODO: Write your code here. And change the return.
    X = df[features]
    y = df['performance']
    root = Node("X", y.mean())
    queue = [CARTStep(X, y, root)]
    while queue:
        curr = queue.pop(0)
        n = len(curr.y)
        candidate_features = ALL_FEATURES - curr.path
        if not candidate_features or n < 3:
            continue
        split = min(split 
                    for split in (
                        Split(curr.X, curr.y, f)
                        for f in sorted(candidate_features)
                    ) if split.is_valid())
        queue.extend(curr.split_by(split))
    return root.as_dict()

# Task 2a: Highest influencing feature

In [29]:
# Write your helper functions here, if needed

In [31]:
def get_highest_influence_feature(cart):
    return cart['split_by_feature']

# Task 2b: Calculate the error rate

In [34]:
# Write your helper functions here, if needed
def get_error_fn(cart): 
    def predict(X: pd.DataFrame) -> float:
        node = cart
        y = cart['mean']
        while (feature := node['split_by_feature']) is not None:
            if X[feature] == 1:
                node = node['successor_left']
            else:
                node = node['successor_right']
            y = node['mean']
        return y
    def sample_error(df: pd.DataFrame) -> float:
        X = df[features]
        y = df['performance']
        return abs(y - predict(X))
    return sample_error

In [48]:
def get_error_rate(cart, sample_set_csv):
    # The sample_set_csv is a file path to a csv data, this can be imported into a dataframe
    df = pd.read_csv(sample_set_csv)
    sample_error = get_error_fn(cart)
    errors = df.aggregate(sample_error, axis="columns")
    return errors.mean()

# Task 2c: Generate optimal configuration

In [ ]:
# Write your helper functions here, if needed

In [ ]:
def get_optimal_configuration(cart, partial_configuration):
    # TODO: Write your code here. And change the return.
    return {"rar", "timestamp"}

# Tests:  
In the following cells, we provide you some test cases (but not all possible test cases!)

In [27]:
# Task 1

test_cart = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'segmentation', 'error_of_split': 6.0, 
             'successor_left': 
                 {'name': 'XL', 'mean': 772.0, 'split_by_feature': 'onegb', 'error_of_split': 0.0, 
                  'successor_left': 
                      {'name': 'XLL', 'mean': 770.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }, 
                  'successor_right': 
                      {'name': 'XLR', 'mean': 773.0, 'split_by_feature': None, 'error_of_split': None, 
                       'successor_left': None, 
                       'successor_right': None
                      }
                 }, 
             'successor_right': 
                 {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                  'successor_left': None, 
                  'successor_right': None}
            }


if get_cart("Performance_01.csv") == test_cart:
    print("passed")
else:
    print("failed")

passed


In [ ]:
# Task 2b
if get_error_rate(test_cart, "Performance_02b.csv") == 5:
    print("passed")
else:
    print("failed")

In [ ]:
# Task 2c
test_cart_v2 = {'name': 'X', 'mean': 763.2, 'split_by_feature': 'zip', 'error_of_split': 0.0, 
                 'successor_left': {'name': 'XL', 'mean': 772.0, 'split_by_feature': None, 'error_of_split': None, 
                                    'successor_left': None, 
                                    'successor_right': None}, 
                 'successor_right': {'name': 'XR', 'mean': 750.0, 'split_by_feature': None, 'error_of_split': None, 
                                     'successor_left': None, 
                                     'successor_right': None}
                }

optimal_config = get_optimal_configuration(test_cart_v2, {"secompress", "encryption", "aes", "algorithm", "signature",
                                                        "timestamp", "segmentation", "onehundredmb"})
reference = {'aes', 'algorithm', 'encryption', 'onehundredmb', 'rar', 'secompress', 'segmentation', 'signature',
            'timestamp'}

if optimal_config == reference:
    print("passed")
else:
    print("failed")